## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.6666,-52.7314,33.76,94,100,19.57,mist
1,1,Yola,NG,9.2000,12.4833,85.08,48,94,2.55,overcast clouds
2,2,Tagusao,PH,9.1924,117.8141,82.22,75,100,6.53,overcast clouds
3,3,Cape Town,ZA,-33.9258,18.4232,64.11,86,0,18.41,clear sky
4,4,Burns Lake,CA,54.2331,-125.7533,31.42,42,100,6.11,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Yola,NG,9.2000,12.4833,85.08,48,94,2.55,overcast clouds
2,2,Tagusao,PH,9.1924,117.8141,82.22,75,100,6.53,overcast clouds
12,12,Rikitea,PF,-23.1203,-134.9692,77.85,71,12,18.34,light rain
13,13,Hilo,US,19.7297,-155.0900,87.28,84,100,5.75,light rain
16,16,Noumea,NC,-22.2763,166.4572,84.18,99,25,8.10,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
678,678,Akyab,MM,20.1500,92.9000,82.80,74,28,6.22,scattered clouds
679,679,Alyangula,AU,-13.8483,136.4192,87.67,70,40,4.61,scattered clouds
682,682,Kombissiri,BF,12.0656,-1.3375,89.58,36,86,15.05,overcast clouds
687,687,Buluang,PH,13.3068,123.3425,83.80,72,87,3.83,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                220
City                   220
Country                220
Lat                    220
Lng                    220
Max Temp               220
Humidity               220
Cloudiness             220
Wind Speed             220
Current Description    220
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yola,NG,85.08,overcast clouds,9.2000,12.4833,
2,Tagusao,PH,82.22,overcast clouds,9.1924,117.8141,
12,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,
13,Hilo,US,87.28,light rain,19.7297,-155.0900,
16,Noumea,NC,84.18,scattered clouds,-22.2763,166.4572,
17,Birao,CF,79.30,broken clouds,10.2849,22.7882,
27,Vaini,TO,84.36,few clouds,-21.2000,-175.2000,
28,Mount Isa,AU,89.37,few clouds,-20.7333,139.5000,
33,Vila Velha,BR,82.35,clear sky,-20.3297,-40.2925,
39,Mumford,GH,80.49,overcast clouds,5.2618,-0.7590,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df = hotel_df.dropna()
hotel_df.isnull().sum()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yola,NG,85.08,overcast clouds,9.2000,12.4833,Muna Hotel
2,Tagusao,PH,82.22,overcast clouds,9.1924,117.8141,Green Villa beach resort
12,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,People ThankYou
13,Hilo,US,87.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
16,Noumea,NC,84.18,scattered clouds,-22.2763,166.4572,Hôtel Le Lagon
...,...,...,...,...,...,...,...
678,Akyab,MM,82.80,scattered clouds,20.1500,92.9000,Hotel Memory
679,Alyangula,AU,87.67,scattered clouds,-13.8483,136.4192,Groote Eylandt Lodge
682,Kombissiri,BF,89.58,overcast clouds,12.0656,-1.3375,Mitibkieta Annexe
687,Buluang,PH,83.80,overcast clouds,13.3068,123.3425,Sacueza Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Wheater</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))